# BNL Development Notebook

This notebook is for developing and testing the BNL package.


In [ ]:
%load_ext autoreload
%autoreload 2

import bnl

# R2_BUCKET_PUBLIC_URL = "https://pub-05e404c031184ec4bbf69b0c2321b98e.r2.dev"

# slm_ds = bnl.data.Dataset(manifest_path=f"{R2_BUCKET_PUBLIC_URL}/manifest_cloud_boolean.csv")
slm_ds = bnl.data.Dataset(manifest_path="~/data/salami/metadata.csv")
track = slm_ds[8]
est = track.load_annotation("adobe-mu1gamma1")
ref = track.load_annotation("reference")

In [ ]:
ref.plot().show()
est.plot().show()

In [ ]:
est.to_contour().plot()